** Final project code **

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import pygsp as pg

In [3]:
from load_and_preprocessing import load_data_and_filter_members, assign_party_to_names
from visualization import label_to_numeric, get_lap_eigendecomp

In [4]:
# Use load_data_and_filter_members to create adjacency for any file from any legislature
# Example 
adjacency, node_index, sum_na_per_row = load_data_and_filter_members('../data/abdb-de-all-affairs-50-0_new.csv',
                                                                     filter_method='number_NA',cutoff=50,ret_transf=False)

(Nbr. of councillors, nbr. of votes) before filter: (434, 1320)
(Nbr. of councillors, nbr. of votes) after filter: (181, 1295)


Assigning parties to the councillors from the dataset loaded above:

In [5]:
name_with_party = assign_party_to_names('../data/Ratsmitglieder_1848_FR.csv', node_index)
print(name_with_party)

         Counc_Id                  CouncillorName PartyAbbreviation
node_id                                                            
0            3907            Thorens Goumaz Adèle               PES
1            3923                       Marra Ada               PSS
2            3897            Moser Tiana Angelina               pvl
3            3902                  Rickli Natalie               UDC
4            3875     Eichenberger-Walther Corina               PLR
5            3876                Estermann Yvette               UDC
6            3877                     Fiala Doris               PRD
7            3878           Flückiger-Bäni Sylvia               UDC
8            3880      Geissbühler Andrea Martina               UDC
9            1346                  Moret Isabelle               PRD
10           3830        Carobbio Guscetti Marina               PSS
11           1129                 Schenker Silvia               PSS
12           1288                    Amherd Viol

Translating the party label into a numerical value:

In [6]:
# Example here is with parties, but the same function works for any kind of dictionary, eg. lobbying mandates
party_map = {'UDC': 6,'PSS': -6,'PDC':0,'BastA':-3,'PLR':5,'pvl':-2,
             'PES':-5, 'PBD':2, 'PdT':-4,'PLS':5,
             'PRD':5, 'MCR':4, 'PEV':-1, 'Lega':3, 'csp-ow':1 }

name_with_party_num, labels_in_data = label_to_numeric(name_with_party, 'PartyAbbreviation', party_map, ret_values=True)

This can be used to see if the label dictionary needs to be adjusted:

In [7]:
print("Unique label values found in data: \n{0}".format(labels_in_data))
print("Label values that are translated by provided dictionary: \n{0}".format(party_map.keys()))

Unique label values found in data: 
['PES' 'PSS' 'pvl' 'UDC' 'PLR' 'PRD' 'PDC' 'PLS' 'PBD' 'PEV' 'Lega'
 'csp-ow' 'MCR' 'BastA' 'PdT']
Label values that are translated by provided dictionary: 
dict_keys(['UDC', 'PSS', 'PDC', 'BastA', 'PLR', 'pvl', 'PES', 'PBD', 'PdT', 'PLS', 'PRD', 'MCR', 'PEV', 'Lega', 'csp-ow'])


Draw some eigenmaps and plot party membership as signal:

In [8]:
eigenvectors = get_lap_eigendecomp(adjacency, lap_type='normalized', ret_eigval=False)

In [9]:
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [10]:
signal = name_with_party_num['PartyAbbreviation'].values
name_labels = name_with_party_num['CouncillorName'].values
colorbar_labels = ['UDC', 'PLR', 'MCR', 'LEGA', 'PBD', 'csp-ow', 'PDC', 'PEV', 'pvl', 'BastA', 'PdT', 'PES', 'PSS']

In [12]:
fig = plt.figure(figsize=(16,9))
ax = fig.add_subplot(111)
ax.scatter(eigenvectors[:,1], eigenvectors[:,2], c=signal, s=15, vmin=-6, vmax=6, cmap="jet")

for i, txt in enumerate(name_labels):
    ax.annotate(txt, (eigenvectors[i,1], eigenvectors[i,2]), xytext=(1,1), textcoords='offset points')
    
cbar = fig.colorbar(ax.collections[0], ticks=np.linspace(6,-6,13), orientation='vertical')
cbar.ax.set_yticklabels(colorbar_labels)
ax.set_title('Position on political spectrum by party ')
fig.tight_layout()